# Classroom Assignment: Estudo de caso para alocação de salas de aula

## Fase 2: Alocação de turmas em salas de aula

Objetivo:
- Maximizar a quantidade de salas alocadas e o número de restrições fracas atendidas

### Modelagem

Conjuntos:

- $C$: Conjunto de salas de aula disponíveis ($c \in C$)
- $Cap_c$: Conjunto de capacidade das salas ($Cap_c \in C$)
- $CT$: Conjunto de tipo de sala disponível ($CT \in C$)
- $CRI$: Conjunto de instituto responsável pela sala ($CRI \in C$). Acrônimo para Classroom Responsable Institute.
- $S$: Conjunto de turmas (s = section) que necessitam de alocação de sala ($s \in S$)
- $SCT$: Conjunto de tipo de sala requerida associado a uma turma ($SCT \in CT$). Acrônimo para Section Classrrom-Type.
- $SRI$: Conjunto de instituto responsável requerida associado a uma turma ($SCT \in CT$). Acrônimo para Section Responsable Institute.
- $SCap_c$: Conjunto de capacidade requerida associado a uma turma ($SCap_c \in CAP_c$). Acrônimo para Section capacity.
- $T_c$: Conjunto de horários (T = Time) das disciplinas ($t \in T_c \in C$)
- $D_c$: Conjunto de dias da semana (D = Day) das disciplinas ($d \in D_c \in C$)
- $S_t$: Subconjunto de turmas que terão seções no horário t ($t \in T_c , S_t \in S$)
- $S_{d}$: Subconjunto de turmas que terão seções no dia da semana d ($d \in D, S_d \in S$)

#### Variáveis

1) Alocação de turma em sala de aula


$$
X_{c,s,d,t} \in (0,1) =

\begin{cases}
1 &  \quad \text{se a sala de aula c for alocada para a turma s para um dia d e um horário t}\\ 
0 & \quad \text{caso contrário} 
\end{cases}

$$

$$
c \in C, s \in S, d \in D_c, t \in T_c
$$

2. Variável de folga (slack variable) que indica a capacidade excedida de lugares que a sala de aula teve ao ter uma turma alocada 

$$
PNC_{c} \in \Z \quad c \in C
$$ 


#### Coeficientes

Considerando 

$$
    c \in C, s \in S, d \in D_c, t \in T_c
$$

temos que:


1) Coeficiente de alocação na sala de aula

    1.1) Sala requerida é a mesma da sala alocada


    1.2) Sala requerida é a mesma da turma a ser alocada E a sala requerida tem o mesmo instituto responsável da turma a ser alocada

    Assim,

    $$

    f_{c,s,d,t} \in (0,1) = 

    \begin{cases}
    10 &  \quad sct_s = ct_c \wedge sri_s = cri_c\\ 
    1 &  \quad sct_s = ct_c\\ 
    0 & \quad \text{caso contrário}
    \end{cases}

    \quad sct_c \in SCT, ct \in CT
    $$

    > Considerando os tipos de salas disponíveis (Anfiteatro, Laboratório, Sala) e o tipo de sala requerido (teórica ou prática), garante que a alocação seja equivalente


2) Fator de penalidade no caso da sala de aula exceda a capacidade requerida na alocação de uma turma 

$$
    Wc = 1000
$$



### Função objetivo

Maximizar a performance acadêmica (EAP, professor habilitado em dar a disciplina) 
<!-- e o número de restrições fracas atendidas -->

$$
MAX \sum_{c \in C}\sum_{s \in S}\sum_{d \in D}\sum_{t \in T} f_{c,s,d,t}X_{c,s,d,t} - Wc \sum_{c \in C} PNC_{c}
$$

$$
    c \in C, s \in S, d \in D_c, t \in T_c
$$

### Restrições

### Restrições fracas

RF1: Garante que a sala seja alocada mesmo se exceder a capacidade da sala. Não inviabiliza o modelo. 


$$
\sum_{c \in C} SCap_c X_{c,s,d,t} \leq Cap_{c} + PNC_{c} \quad c \in C
$$


##### Restrições fortes

RH1: Um sala poderá ser alocada para no máximo 1 uma turma em um mesmo dia e horário (binário)


$$
\sum_{c \in C}\sum_{s \in (S_t \cap S_{d})} X_{c,s,d,t} \leq 1  \quad d \in D_c, t \in T_c 
$$

RH2: Uma seção deverá ter somente uma sala de aula

$$
\sum_{c \in C} X_{c,s,d,t} = 1  \quad d \in D_c, t \in T_c 
$$



